<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Bispectrum-counter-example" data-toc-modified-id="Bispectrum-counter-example-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Bispectrum counter example</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#optimize-structures" data-toc-modified-id="optimize-structures-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>optimize structures</a></span></li></ul></li><li><span><a href="#Feature-computation" data-toc-modified-id="Feature-computation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Feature computation</a></span><ul class="toc-item"><li><span><a href="#$\rho_{i}^{\otimes-3}$" data-toc-modified-id="$\rho_{i}^{\otimes-3}$-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>$\rho_{i}^{\otimes 3}$</a></span></li><li><span><a href="#check-map-between-three-center-and-single-center" data-toc-modified-id="check-map-between-three-center-and-single-center-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>check map between three center and single center</a></span></li><li><span><a href="#check-degeneracy-of-$\rho_{i}^{\otimes-3}$" data-toc-modified-id="check-degeneracy-of-$\rho_{i}^{\otimes-3}$-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>check degeneracy of $\rho_{i}^{\otimes 3}$</a></span></li><li><span><a href="#artificial-structure" data-toc-modified-id="artificial-structure-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>artificial structure</a></span></li></ul></li><li><span><a href="#encoder-decoder" data-toc-modified-id="encoder-decoder-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>encoder-decoder</a></span><ul class="toc-item"><li><span><a href="#train-different-blocks-separately" data-toc-modified-id="train-different-blocks-separately-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>train different blocks separately</a></span></li><li><span><a href="#tensormap-of-reconstructed-features" data-toc-modified-id="tensormap-of-reconstructed-features-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>tensormap of reconstructed features</a></span></li></ul></li><li><span><a href="#set-up-NICE-style-feature-computation" data-toc-modified-id="set-up-NICE-style-feature-computation-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>set up NICE style feature computation</a></span></li><li><span><a href="#encoder-decoder-like-architecture---nope" data-toc-modified-id="encoder-decoder-like-architecture---nope-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>encoder-decoder like architecture - nope</a></span><ul class="toc-item"><li><span><a href="#$\rho_{i}^{\otimes-3}$" data-toc-modified-id="$\rho_{i}^{\otimes-3}$-1.4.1"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>$\rho_{i}^{\otimes 3}$</a></span></li><li><span><a href="#$\rho_{i-i_1-i_2}^{\otimes-1}$" data-toc-modified-id="$\rho_{i-i_1-i_2}^{\otimes-1}$-1.4.2"><span class="toc-item-num">1.4.2&nbsp;&nbsp;</span>$\rho_{i i_1 i_2}^{\otimes 1}$</a></span></li><li><span><a href="#check-degeneracy-of-$\rho_{i}^{\otimes-3}$" data-toc-modified-id="check-degeneracy-of-$\rho_{i}^{\otimes-3}$-1.4.3"><span class="toc-item-num">1.4.3&nbsp;&nbsp;</span>check degeneracy of $\rho_{i}^{\otimes 3}$</a></span></li></ul></li></ul></li></ul></div>

# Bispectrum counter example

In [1]:
%load_ext autoreload
%autoreload 2 

In [7]:
from ase.io import read,write
from ase.visualize import view
import numpy as np 

from equistore import Labels, TensorBlock, TensorMap
from itertools import product
from utils.clebsh_gordan import ClebschGordanReal

import matplotlib.pyplot as plt

from utils.librascal import  RascalSphericalExpansion, RascalPairExpansion
from rascal.representations import SphericalExpansion
import copy
import ase 
import tqdm 

In [8]:
from utils.mp_utils import *

In [9]:
from generate_bispectrum_structures import generate_nu3_degen_structs, analytical_partial_derivative

In [4]:
import pyscf
from pyscf import gto
from pyscf import scf,dft

#gto.M default unit being Angstrom
def pyscf_rks_calc(frame):
    pts = frame.get_positions()
    els = frame.get_chemical_symbols()
    natoms = len(pts)
    atoms = []
    for i in range(natoms):
        coord = pts[i]
        atoms.append([els[i],(coord[0],coord[1],coord[2])])
    mol = gto.M(atom=atoms,symmetry = True)
    mol.basis = "ccpvdz" #"def2-SVP" 
#     mol.basis = {'O': 'ccpvdz', 'H': 'ccpvdz', 'C': 'ccpvdz', 'N':'ccpvdz', 'Ag': 'def2-svp'} 
    mol.charge=0
    mol.spin = 0
    mol.build()
    m = scf.RKS(mol)
#     m = scf.RHF(mol)
    m.xc="pbe"
    m.max_cycle=600
    m.conv_tol=1e-9
    m.conv_tol_grad=1e-8
    m.diis_space=10
#     print(mol.ao_labels())

    return m.kernel() 

def pyscf_rhf_calc(frame):
    pts = frame.get_positions()
    els = frame.get_chemical_symbols()
    natoms = len(pts)
    atoms = []
    for i in range(natoms):
        coord = pts[i]
        atoms.append([els[i],(coord[0],coord[1],coord[2])])
    mol = gto.M(atom=atoms,symmetry = True)
    mol.basis = "ccpvdz" #"def2-SVP" 
#     mol.basis = {'O': 'ccpvdz', 'H': 'ccpvdz', 'C': 'ccpvdz', 'N':'ccpvdz', 'Ag': 'def2-svp'} 
    mol.charge=0
    mol.spin = 0
    mol.build()
#     m = scf.RKS(mol)
    m = scf.RHF(mol)
#     m.xc="pbe"
    m.max_cycle=500
    m.conv_tol=1e-9
#     m.conv_tol_grad=1e-8
    m.diis_space=10
#     print(mol.ao_labels())

    return m.kernel() 

from pyscf.pbc.tools.pyscf_ase import atoms_from_ase
def dftblock(frames, start, end, basis='ccpvdz', xc='pbe'):
#     frames = read(prefix+".xyz", "%d:%d" % (start,end))
    idx_nconv=[]
    for i, f in enumerate(tqdm.tqdm(frames[start:end])):
        if "label" in f.info:
            label = f.info["label"]
        else:
            label = str(start+i)    
        f.info = {}
        mol = pyscf.gto.M(atom = atoms_from_ase(f), basis = basis)
        rks = pyscf.dft.RKS(mol)
        rks = pyscf.scf.addons.smearing_(rks, sigma=.01, method='fermi')
        rks.xc = xc  
        rks.grids.level = 5 
        rks.verbose = 3 
        rks.init_guess = 'minao'
        rks.scf(print=True) 
        try:
            assert rks.converged==True
            print("converged")
        except: 
            print(i)
            idx_nconv.append(start+i)
        dft = rks.e_free # energy_tot()
        ddft = rks.nuc_grad_method().grad()
        f.info["energy_ha"] = dft 
        f.arrays["gradient_habohr"] = ddft
        f.info["label"] = label
        f.info["basis"] = basis
        f.info["xc"] = xc
    return frames, idx_nconv

pyscf plugins found in 
/Users/jigyasa/scratch/pyscf
/Users/jigyasa/scratch/./pyscf
/Users/jigyasa/miniconda3/lib/python3.8/site-packages/pyscf
When PYTHONPATH is set, it is recommended to load these plugins through the environment variable PYSCF_EXT_PATH


In [11]:
import pyscf
from pyscf import gto
from pyscf import scf,dft

def pyscf_rks_calc(frame):
    pts = frame.get_positions()
    els = frame.get_chemical_symbols()
    natoms = len(pts)
    atoms = []
    for i in range(natoms):
        coord = pts[i]
        atoms.append([els[i],(coord[0],coord[1],coord[2])])
    mol = gto.M(atom=atoms,symmetry = True)
    mol.basis = "ccpvdz" #"def2-SVP" 
#     mol.basis = {'O': 'ccpvdz', 'H': 'ccpvdz', 'C': 'ccpvdz', 'N':'ccpvdz', 'Ag': 'def2-svp'} 
    mol.charge=0
    mol.spin = 0
    mol.build()
    m = scf.RKS(mol)
#     m = scf.RHF(mol)
    m.xc="pbe"
    m.max_cycle=600
    m.conv_tol=1e-9
    m.conv_tol_grad=1e-8
    m.diis_space=10
#     print(mol.ao_labels())

    return m.kernel() 

def pyscf_rhf_calc(frame):
    pts = frame.get_positions()
    els = frame.get_chemical_symbols()
    natoms = len(pts)
    atoms = []
    for i in range(natoms):
        coord = pts[i]
        atoms.append([els[i],(coord[0],coord[1],coord[2])])
    mol = gto.M(atom=atoms,symmetry = True)
    mol.basis = "ccpvdz" #"def2-SVP" 
#     mol.basis = {'O': 'ccpvdz', 'H': 'ccpvdz', 'C': 'ccpvdz', 'N':'ccpvdz', 'Ag': 'def2-svp'} 
    mol.charge=0
    mol.spin = 0
    mol.build()
#     m = scf.RKS(mol)
    m = scf.RHF(mol)
#     m.xc="pbe"
    m.max_cycle=500
    m.conv_tol=1e-9
#     m.conv_tol_grad=1e-8
    m.diis_space=10
#     print(mol.ao_labels())

    return m.kernel() 

/scratch/nigam/anaconda3/envs/py3.7/lib/python3.8/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [14]:
# Advice on choosing the basis sets for different calculations 

#https://sites.google.com/site/orcainputlibrary/basis-sets?pli=1

In [7]:
np.random.seed(12345)
ngen = 1000
frames = []
for f in tqdm.tqdm(range(ngen)):
#     print(i)
    for i in range(4000):
        r = np.random.uniform(0.5,2)
        z1 = np.random.uniform(0.5,2)
        z2 = np.random.uniform(1.5,1.8)
        psi = np.random.uniform(0,np.pi)
        phi1 = np.random.uniform(np.pi/6, np.pi)
        phi2 = np.random.uniform(np.pi/6, np.pi)
        fr = generate_nu3_degen_structs(r, [0, phi1, phi1+phi2], 
                                   psi, z1, z2, "B", "B", "B")
        dist = fr[0].get_all_distances()
        if dist[0,1:].min()>1.5 and dist[0,1:].max()<1.8 and dist[np.triu_indices(len(dist),1)].flatten().min()>1.2:
            fr[0].info["pars"] = str((f, "+")+(r, z1, z2, psi, phi1, phi2))
            fr[1].info["pars"] = str((f, "-")+(r, z1, z2, psi, phi1, phi2))
#             print(i)
            break
    if (i>3990):
        print("oops!")
    frames += fr

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 191.28it/s]


In [ ]:
# dft_frames = dftblock(frames, 0, 4000, basis='ccpvdz', xc='pbe')

  0%|                                                                                          | 0/4000 [00:00<?, ?it/s]

converged SCF energy = -197.783716100935
converged
--------------- RKS gradients ---------------
         x                y                z
0 B     0.0497713870    -0.0461787860     0.0479342187
1 B    -0.0798080939     0.1003805258    -0.2079702769
2 B     0.1169919658     0.0900814024    -0.3259787067
3 B     0.1280291256     0.0589133385    -0.2539272829
4 B    -0.0749616261    -0.1447615299     0.2942151973
5 B    -0.1430563662     0.0505071919     0.2155812910
6 B     0.0013580838    -0.1182632336     0.1902667050
7 B     0.0016745295     0.0093209641     0.0398785948
----------------------------------------------


  0%|                                                                               | 1/4000 [00:28<32:06:23, 28.90s/it]

converged SCF energy = -197.783482709666
converged
--------------- RKS gradients ---------------
         x                y                z
0 B     0.0625976524    -0.0294848959    -0.0469110138
1 B    -0.0830730338     0.1129955667    -0.1725864386
2 B     0.1239634930     0.1036300094    -0.2934280323
3 B     0.1221478201     0.0284010864    -0.2343122852
4 B    -0.0615192577    -0.1345194398     0.3239845306
5 B    -0.1286409846     0.0508534005     0.2532857458
6 B    -0.0260913780    -0.1280346519     0.2096322546
7 B    -0.0093853988    -0.0038412702    -0.0396649214
----------------------------------------------


  0%|                                                                               | 2/4000 [00:56<31:14:12, 28.13s/it]

...

converged SCF energy = -197.871230555705
converged
--------------- RKS gradients ---------------
         x                y                z
0 B    -0.0338384760     0.0062506420     0.1353545917
1 B     0.0333704842    -0.0158288618     0.0222768237
2 B     0.0079979612     0.0235555374     0.0244280225
3 B    -0.0203642516    -0.0092121376     0.0294017295
4 B     0.0129033782     0.0281624447    -0.0371143720
5 B    -0.0304077685     0.0007699083    -0.0441839758
6 B     0.0282794929    -0.0293387008    -0.0285308660
7 B     0.0020585797    -0.0043588791    -0.1016305930
----------------------------------------------


  0%|▎                                                                             | 17/4000 [07:49<32:05:10, 29.00s/it]

converged SCF energy = -197.871507701584
converged
--------------- RKS gradients ---------------
         x                y                z
0 B    -0.0326876611     0.0077804935    -0.1365048368
1 B     0.0282509411    -0.0181773208     0.0311457757
2 B     0.0081010587     0.0365306043     0.0345973497
3 B    -0.0239795048    -0.0231026673     0.0449080410
4 B     0.0111408904     0.0300278907    -0.0245199773
5 B    -0.0153172402    -0.0087005411    -0.0305633374
6 B     0.0193306047    -0.0267535518    -0.0204694247
7 B     0.0051610129     0.0023954790     0.1014066377
----------------------------------------------


  0%|▎                                                                             | 18/4000 [08:17<31:44:32, 28.70s/it]

converged SCF energy = -197.825089223645
converged


### optimize structures 

In [55]:
def get_params(frame):
    parameters = []
    for x in frame.info["pars"].split()[2:]:
        parameters.append(float(x[:-1]))
    return parameters

In [2]:
delta_r = np.linspace(-0.5, 0.5, 10)

f = frames[0].copy()
for dr in delta_r: 
    r, z1, z2, psi, phi1, phi2 = get_params(f)
    par = [r+dr, [0,phi1, phi2], psi, z1, z2, 'B','B','B' ]
    fr = generate_nu3_degen_structs(r+dr, [0, phi1, phi1+phi2], 
                                   psi, z1, z2, "B", "B", "B")
    en_plus = pyscf_rhf_calc(fr[0])
    en_minus = pyscf_rhf_calc(fr[1])
    delta_e = abs(en_plus - en_minus)
    print(delta_e)
    der = analytical_partial_derivative(par, diff_param= 'r' )[0]
    
#     print(der)

## Feature computation

In [8]:
from rascaline import SphericalExpansion
from rascaline import SphericalExpansionByPair as PairExpansion
from equistore import operations

In [9]:
def simplify_gij(gij):
    keys = list(gij.keys.dtype.names)
    if "species_atom_1" in keys: 
        keys[keys.index('species_atom_1')] = "species_center"
    if "species_atom_2" in keys: 
        keys[keys.index('species_atom_2')] = "species_neighbor"
    newkeys = Labels(keys, gij.keys.asarray())
    block_list = []
    for k,b in gij: 
        block = TensorBlock(
        values = b.values, 
        components = b.components, 
        properties = b.properties,
        samples = Labels(["structure", "center", "neighbor"] , np.asarray(b.samples.tolist())[:,[0,2,3]])
        )
        block_list.append(block)
    
    return TensorMap(newkeys, block_list)

In [10]:
hypers = {
    "cutoff": 3.5,
    "max_radial": 4,
    "max_angular": 3,
    "atomic_gaussian_width": 0.2,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.1}},
    "center_atom_weight": 0.0,    
}
cg = ClebschGordanReal(5)

calculator = SphericalExpansion(**hypers)
rhoi = calculator.compute(frames)
rhoi = rhoi.keys_to_properties(['species_neighbor'])
rho1i = acdc_standardize_keys(rhoi)
    


# selects only one environment
rho1i = operations.slice(rho1i, samples=Labels(['center'],np.array([[0]], np.int32)) )
# norm_rho1 = np.sqrt(np.sum([(b.values**2).sum(axis=(1,2)) for b in rho1i.blocks()],axis=0).mean())
# for b in rho1i.blocks():
#     b.values[:]/=norm_rho1
    

calculator = PairExpansion(**hypers)
gij = calculator.compute(frames)
gij = operations.slice(gij, samples=Labels(['first_atom'],np.array([[0]], np.int32)) )
gij =  acdc_standardize_keys(gij)
# for b in gij.blocks():
#     b.values[:]/=norm_rho1
gij = simplify_gij(gij)

In [11]:
test = operations.sum_over_samples(gij, samples_names=["neighbor"]) 

np.linalg.norm(rho1i.block(0).values/test.block(0).values -1)

0.0

### $\rho_{i}^{\otimes 3}$

In [14]:
rho2i = cg_increment(rho1i, rho1i, clebsch_gordan=cg, lcut=3, other_keys_match=["species_center"])
rho3i = cg_increment(rho2i, rho1i, clebsch_gordan=cg, lcut=0, other_keys_match=["species_center"])
# rho4i = cg_increment(rho3i, rho1i, clebsch_gordan=cg, lcut=3, other_keys_match=["species_center"])
# rho5i = cg_increment(rho4i, rho1i, clebsch_gordan=cg, lcut=0, other_keys_match=["species_center"])

In [395]:
xx =  cg_increment(rho1i, rho1i, clebsch_gordan=cg, lcut=3, other_keys_match=["species_center"])
for k, b in rho2i: 
    nu, s, l, spa = k
    idx = xx.blocks_matching(inversion_sigma=s, spherical_harmonics_l=l)[0]
    xblock = xx.block(idx)
    print(k, np.linalg.norm(b.values-xblock.values))

(2, 1, 0, 5) 0.0
(2, 1, 1, 5) 0.0
(2, 1, 2, 5) 0.0
(2, 1, 3, 5) 0.0
(2, -1, 1, 5) 0.0
(2, -1, 2, 5) 0.0
(2, -1, 3, 5) 0.0


In [12]:
rhoii1i2_nu0 = cg_combine(gij, gij, clebsch_gordan=cg, other_keys_match=['species_center'], lcut=3)
rhoii1i2_nu1 =  cg_combine(rho1i, rhoii1i2_nu0, clebsch_gordan=cg, other_keys_match = ['species_center'], lcut=0)

### check map between three center and single center 

In [397]:
rho2i_full = cg_combine(rho1i, rho1i, clebsch_gordan=cg, lcut=3, sorted_l=False)
test = operations.sum_over_samples(rhoii1i2_nu0, samples_names=["neighbor_1", "neighbor_2"]) 

In [398]:
np.linalg.norm(rho2i_full.block(2).values/test.block(2).values-1)

2.3955878289546457e-12

### check degeneracy of $\rho_{i}^{\otimes 3}$

In [15]:
np.linalg.norm(rho3i.block(spherical_harmonics_l=0,inversion_sigma=1 ).values[0] - rho3i.block(spherical_harmonics_l=0,inversion_sigma=1 ).values[1])


1.7279430568860854e-17

In [14]:
# rhoii1i2_nu1.keys_to_properties('species_neighbor_1')
# rhoii1i2_nu1.keys_to_properties('species_neighbor_2')

In [16]:
rhoii1i2_nu1.block(inversion_sigma=1).samples[:64]#values.shape


Labels([(0, 0, 0, 0), (0, 0, 0, 1), (0, 0, 0, 2), (0, 0, 0, 3),
        (0, 0, 0, 4), (0, 0, 0, 5), (0, 0, 0, 6), (0, 0, 0, 7),
        (0, 0, 1, 0), (0, 0, 1, 1), (0, 0, 1, 2), (0, 0, 1, 3),
        (0, 0, 1, 4), (0, 0, 1, 5), (0, 0, 1, 6), (0, 0, 1, 7),
        (0, 0, 2, 0), (0, 0, 2, 1), (0, 0, 2, 2), (0, 0, 2, 3),
        (0, 0, 2, 4), (0, 0, 2, 5), (0, 0, 2, 6), (0, 0, 2, 7),
        (0, 0, 3, 0), (0, 0, 3, 1), (0, 0, 3, 2), (0, 0, 3, 3),
        (0, 0, 3, 4), (0, 0, 3, 5), (0, 0, 3, 6), (0, 0, 3, 7),
        (0, 0, 4, 0), (0, 0, 4, 1), (0, 0, 4, 2), (0, 0, 4, 3),
        (0, 0, 4, 4), (0, 0, 4, 5), (0, 0, 4, 6), (0, 0, 4, 7),
        (0, 0, 5, 0), (0, 0, 5, 1), (0, 0, 5, 2), (0, 0, 5, 3),
        (0, 0, 5, 4), (0, 0, 5, 5), (0, 0, 5, 6), (0, 0, 5, 7),
        (0, 0, 6, 0), (0, 0, 6, 1), (0, 0, 6, 2), (0, 0, 6, 3),
        (0, 0, 6, 4), (0, 0, 6, 5), (0, 0, 6, 6), (0, 0, 6, 7),
        (0, 0, 7, 0), (0, 0, 7, 1), (0, 0, 7, 2), (0, 0, 7, 3),
        (0, 0, 7, 4), (0, 0, 7, 5), (0, 

In [17]:
np.linalg.norm(rhoii1i2_nu1.block(inversion_sigma=1 ).values[:64])

0.1273083301388986

In [18]:
# diff = np.sum(rhoii1i2_nu1.block(species_center = 6).values[:64], axis=0) - np.sum(rhoii1i2_nu1.block(species_center = 6).values[64:], axis=0)
diff = rhoii1i2_nu1.block(inversion_sigma=1).values[:64] - rhoii1i2_nu1.block(inversion_sigma=1).values[64:128]

np.linalg.norm(diff)

0.06589105111408992

In [19]:
# from equistore import operations
# ntrain = 400
# train_tmap = operations.slice(rhoii1i2_nu1, 
#                 samples = Labels(
#             names=["structure"],
#             values=np.array(
#                 list(range(ntrain))
#             ).reshape(-1,1), )
#                 )

# test_tmap = operations.slice(rhoii1i2_nu1, 
#                 samples = Labels(
#             names=["structure"],
#             values=np.array(
#                 list(range(ntrain, 500))
#             ).reshape(-1,1), )
#                 )

# from equistore.io import save, load
# # save('./rho3i_400.npz', rh)
# save('./rhoii1i2n1_400.npz', train_tmap)
# save('./rhoii1i2n1_400-500.npz', test_tmap)

/Users/jigyasa/miniconda3/lib/python3.8/site-packages/equistore/operations/slice.py:113: UserWarning: Some TensorBlocks in the sliced TensorMap are now empty, based on your choice of samples and/or properties to slice by. The keys of the empty TensorBlocks are:
 [(3, 1, 0,  1,  1, 12) (3, 1, 0,  1,  3, 12) (3, 1, 0,  1, 12,  1)
 (3, 1, 0,  1, 12,  3) (3, 1, 0,  1, 12, 12) (3, 1, 0,  3,  1, 12)
 (3, 1, 0,  3,  3, 12) (3, 1, 0,  3, 12,  1) (3, 1, 0,  3, 12,  3)
 (3, 1, 0,  3, 12, 12) (3, 1, 0, 12,  1,  1) (3, 1, 0, 12,  1,  3)
 (3, 1, 0, 12,  1, 12) (3, 1, 0, 12,  3,  1) (3, 1, 0, 12,  3,  3)
 (3, 1, 0, 12,  3, 12) (3, 1, 0, 12, 12,  1) (3, 1, 0, 12, 12,  3)]
  warnings.warn(


### artificial structure

In [372]:
from generate_bispectrum_structures import * 
from ase import Atoms

In [373]:
center_species = ring_species= z2species ='B'
n=2

phi1 = np.pi/4 #np.random.uniform(np.pi/6, np.pi)
phi2 = -np.pi/3 #np.random.uniform(np.pi/6, np.pi)
r=2
z1=0
z2 = 4
layer1 = points_on_circle(r,[phi1,phi2])

positions = np.zeros((4,3))
positions[1:3,:2] = layer1
positions[1:3, 2] = z1
# positions[1,:2] = [0,0]
positions[3,:2] = [np.random.uniform(0.2,1.2),np.random.uniform(0.2,1.2)]
positions[3, 2] =z2


atom_string = center_species + ring_species*(n) + z2species
atoms = Atoms(atom_string, positions=positions[:], pbc=False)

positions2 = positions.copy()
positions2[2:4, 0] = -positions[2:4,0]
positions2[2:4, 1] = positions[2:4, 1]
atoms2 = Atoms(atom_string, positions=positions2[:], pbc=False)

positions3 = positions.copy()
positions3[3,2]=-4
atoms3 = Atoms(atom_string, positions=positions3[:], pbc=False)

In [374]:
print(atoms.get_all_distances() )

# print(atoms2.get_all_distances() ) # is equal to atoms when the first two atoms are non complanar with the origin

print(atoms3.get_all_distances()) # inverter structure basically 

[[0.         2.         2.         4.13338024]
 [2.         0.         3.17341336 4.11323662]
 [2.         3.17341336 0.         4.71064777]
 [4.13338024 4.11323662 4.71064777 0.        ]]
[[0.         2.         2.         4.13338024]
 [2.         0.         3.17341336 4.11323662]
 [2.         3.17341336 0.         4.71064777]
 [4.13338024 4.11323662 4.71064777 0.        ]]


In [375]:
view([atoms, atoms3])

In [376]:
artificial = [atoms, atoms3] #atoms2
calculator = SphericalExpansion(**hypers)
ar_rhoi = calculator.compute(artificial)
ar_rhoi = ar_rhoi.keys_to_properties(['species_neighbor'])
ar_rho1i = acdc_standardize_keys(ar_rhoi)
# selects only one environment
ar_rho1i = operations.slice(ar_rho1i, samples=Labels(['center'],np.array([[0]], np.int32)) )
# norm_rho1 = np.sqrt(np.sum([(b.values**2).sum(axis=(1,2)) for b in rho1i.blocks()],axis=0).mean())
# for b in rho1i.blocks():
#     b.values[:]/=norm_rho1
ar_rho2i = cg_increment(ar_rho1i, ar_rho1i, clebsch_gordan=cg, lcut=3, other_keys_match=["species_center"])
ar_rho3i = cg_increment(ar_rho2i, ar_rho1i, clebsch_gordan=cg, lcut=0, other_keys_match=["species_center"])  

calculator = PairExpansion(**hypers)
ar_gij = calculator.compute(artificial)
ar_gij = operations.slice(ar_gij, samples=Labels(['first_atom'],np.array([[0]], np.int32)) )
ar_gij =  acdc_standardize_keys(ar_gij)
# for b in gij.blocks():
#     b.values[:]/=norm_rho1
ar_gij = simplify_gij(ar_gij)
ar_rhoii1i2_nu0 = cg_combine(ar_gij, ar_gij, clebsch_gordan=cg, other_keys_match=['species_center'], lcut=3)
ar_rhoii1i2_nu1 =  cg_combine(ar_rho1i, ar_rhoii1i2_nu0, clebsch_gordan=cg, other_keys_match = ['species_center'], lcut=0)


In [377]:
ar_rhoii1i2_nu1.block(inversion_sigma=-1).values.squeeze()[:16] -ar_rhoii1i2_nu1.block(inversion_sigma=-1).values.squeeze()[16:32]


array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-3.70477971e-06, -2.94558474e-04, -4.58446563e-07, ...,
         3.16165055e-05,  4.93809604e-08,  3.93677329e-06],
       [ 3.70477971e-06,  2.94558474e-04,  4.58446563e-07, ...,
        -3.16165055e-05, -4.93809604e-08, -3.93677329e-06],
       [-8.78154690e-25,  5.62335965e-23, -6.35203908e-23, ...,
        -1.60461975e-23,  3.18368910e-23, -8.55700383e-22]])

In [380]:
np.where((ar_rho3i.block(inversion_sigma = -1).values[0] - ar_rho3i.block(inversion_sigma = -1).values[1])>1e-7)

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([10, 11, 20, 21, 25, 29, 33, 35, 42, 46, 52, 53]))

## encoder-decoder

encode three center features, sum over i1,i2 and then decode 

In [19]:
import torch
def StructureMap(samples_structure, device="cpu"):
    unique_structures, unique_structures_idx = np.unique(
        samples_structure, return_index=True
    )
    new_samples = samples_structure[np.sort(unique_structures_idx)]
    # we need a list keeping track of where each atomic contribution goes
    # (e.g. if structure ids are [3,3,3,1,1,1,6,6,6] that will be stored as
    # the unique structures [3, 1, 6], structure_map will be
    # [0,0,0,1,1,1,2,2,2]
    replace_rule = dict(zip(unique_structures, range(len(unique_structures))))
    structure_map = torch.tensor(
        [replace_rule[i] for i in samples_structure],
        dtype=torch.long,
        device=device,
    )
    return structure_map, new_samples, replace_rule

In [20]:
def get_triple_samples(bsamples, train_idx):
    samples = []
    for i in trainidx:
        loc = np.where(bsamples["structure"] == i)
        samples.append(bsamples[loc].asarray())
    return Labels(rhoii1i2_nu1.sample_names, np.asarray(np.concatenate(samples), dtype=np.int32))

In [21]:
device='cpu'
torch.set_default_dtype(torch.float64)

In [22]:
class BlockEncoderDecoder(torch.nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.encode_type = kwargs['encode_type']
        
        if self.encode_type == 'linear':
            self.encoder_output_layer = torch.nn.Linear(
            in_features=kwargs["input_size"], out_features=kwargs["intermediate_size"]
        )
        
        else: 
            self.encoder_hidden_layer = torch.nn.Linear(
                in_features=kwargs["input_size"], out_features=kwargs["hidden_size"]
            )
            
            self.encoder_output_layer = torch.nn.Linear(
                in_features=kwargs["hidden_size"], out_features=kwargs["intermediate_size"]
            )
        
        
        self.decoder_hidden_layer = torch.nn.Linear(
            in_features=kwargs["intermediate_size"], out_features=kwargs["hidden_size"]
        )
        if 'output_size' in kwargs: 
            self.decoder_output_layer = torch.nn.Linear(
            in_features=kwargs["hidden_size"], out_features=kwargs["output_size"]
        )
        else: 
            self.decoder_output_layer = torch.nn.Linear(
            in_features=kwargs["hidden_size"], out_features=kwargs["input_size"]
            )
        
        self.sum_triples=None
        self.samples = kwargs["triple_samples"]
        self.to(device)
        
        
    
    def structure_wise_feats(self, x): 
        if self.encode_type == 'linear': 
            x = torch.from_numpy(x)
            encoded = self.encoder_output_layer(x)
#             print("encoded_shape", encoded.shape)
        
        else: 
            x = torch.from_numpy(x.values)
            encoded = self.encoder_hidden_layer(x)
            activation = torch.tanh(encoded)
            encoded = self.encoder_output_layer(activation)
            encoded = torch.tanh(encoded)
#             print("encoded_shape", encoded.shape)
        
        structure_map, new_samples, _ = StructureMap(
            self.samples["structure"], "cpu"
        )
        self.sum_triples =  torch.zeros((len(new_samples), encoded.shape[-1]), device=x.device)
        self.sum_triples.index_add_(0, structure_map, encoded)
        
        return self.sum_triples
    
    def forward(self, x):
#         print("from block forward", x.keys)
        activation = self.decoder_hidden_layer(x)
        activation = torch.tanh(activation)
        activation = self.decoder_output_layer(activation)
        decoded = torch.tanh(activation)
#         print("decoded_shape", decoded.shape)
        return decoded
    

class ThreeCenterModel(torch.nn.ModuleDict):
    
    def __init__(self, feature, train_idx, hidden_size=128, intermediate_size=128, encode_type='linear'):
        super().__init__()
        self.feature = feature
        self.encode_type=encode_type
        self.models = torch.nn.ModuleDict()
        self.trainidx = train_idx #idx of train structures
        for k, b in self.feature:
            input_size = b.values.shape[-1]
            print(k,input_size, hidden_size, intermediate_size)
#             samples = get_triple_samples(b.samples, self.trainidx)
            self.models[str(k)] = BlockEncoderDecoder(input_size=input_size, hidden_size=hidden_size, intermediate_size=intermediate_size, 
                                                      encode_type=self.encode_type, triple_samples = b.samples
                                                     )            
        
        self.to(device)
    
    def forward(self, feature):
        pred_blocks =[]
        for k, block in feature:

            struct_feat = self.models[str(k)].structure_wise_feats(block.values.squeeze())
            structure_map, new_samples, _ = StructureMap(block.samples["structure"], device)
        
            recons = self.models[str(k)].forward(struct_feat) 
#             print(k, recons.shape,recons.reshape((recons.shape[0], recons.shape[1], 1)).swapaxes(1,-1).shape)
#         return recons
            newblock = TensorBlock(
                        values=recons,#.reshape((recons.shape[0], recons.shape[1], 1)).swapaxes(1,-1),
                        samples=Labels(["structure"], new_samples.asarray()),
                        components=[],#block.components,
                        properties= Labels(["prop"], np.asarray([i for i in range(recons.shape[1])], dtype=np.int32).reshape(-1,1))
                    )
            pred_blocks.append(newblock) 
        
        keys = feature.keys
        pred_target = TensorMap(keys, pred_blocks)
        return(pred_target)
    
    


### train different blocks separately 

In [23]:
allidx = range(1000*2)

trainidx = range(900*2)
testidx = list(set(allidx).difference(trainidx))

In [24]:
block = rhoii1i2_nu1.block(0)

In [25]:
target = torch.from_numpy(rho3i.block(spherical_harmonics_l = 0, inversion_sigma = 1).values.squeeze())

In [35]:
input_size = block.values.shape[-1]
hidden_size = 256
intermediate_size = 256
encode_type = 'linear'
output_size = target.shape[-1]
model_block = BlockEncoderDecoder(input_size=input_size, hidden_size=hidden_size, 
                                  intermediate_size=intermediate_size, encode_type=encode_type, 
                                  triple_samples = block.samples, output_size = output_size
                                
                                                     )            
struct_feat = model_block.structure_wise_feats(block.values.squeeze())        
recons = model_block.forward(struct_feat) 
            

In [43]:
def mse_loss(pred, target):
    return torch.sum((pred - target)**2)

In [44]:
optimizer = torch.optim.LBFGS(
        model_block.parameters(),
        lr=0.1,
        line_search_fn="strong_wolfe",
        history_size=256,
    )

In [ ]:
nepochs = 200

for epoch in range(nepochs):
    
    def single_step():
        optimizer.zero_grad()
        loss = torch.zeros(size=(1,), device=device)
        struct_feat = model_block.structure_wise_feats(block.values.squeeze())
        predicted = model_block(struct_feat)
        loss += mse_loss(predicted[trainidx], target[trainidx])
        
        loss.backward(retain_graph=False)
        return loss

    loss = optimizer.step(single_step)
#     print(loss)
    struct_feat = model_block.structure_wise_feats(block.values.squeeze())
    predicted = model_block(struct_feat) #, triplet_samples["structure"][ntrain*ntriples:])
    
#     if epoch % 5 ==0:
    print("Epoch:", epoch, "RMSE: train ", np.sqrt(loss.detach().numpy().flatten()[0]/len(trainidx)),
     "test", np.sqrt(mse_loss(predicted[testidx], target[testidx]).detach().numpy().flatten()[0]/ len(testidx))
         )

Epoch: 0 RMSE: train  1.2078184157107028 test 0.9727308204177948
Epoch: 1 RMSE: train  0.9731485804210459 test 0.7692942280219645
Epoch: 2 RMSE: train  0.770147507949278 test 0.2847334423761676
Epoch: 3 RMSE: train  0.2865970419606375 test 0.14087041917183318
Epoch: 4 RMSE: train  0.14446736011405278 test 0.11320443720198393


### tensormap of reconstructed features

In [618]:
model = ThreeCenterModel(rhoii1i2_nu1, train_idx = [0,1,3,5], encode_type='linear')

(3, 1, 0, 5, 5, 5) 184 128 128
(3, -1, 0, 5, 5, 5) 88 128 128


In [619]:
model(rhoii1i2_nu1)

encoded_shape torch.Size([640, 128])
decoded_shape torch.Size([10, 184])
[(0,) (1,) (2,) (3,) (4,) (5,) (6,) (7,) (8,) (9,)]
encoded_shape torch.Size([640, 128])
decoded_shape torch.Size([10, 88])
[(0,) (1,) (2,) (3,) (4,) (5,) (6,) (7,) (8,) (9,)]


TensorMap with 2 blocks
keys: ['order_nu' 'inversion_sigma' 'spherical_harmonics_l' 'species_center' 'species_neighbor_a' 'species_neighbor_b']
           3             1                    0                   5                 5                   5
           3            -1                    0                   5                 5                   5

In [682]:
block = rhoii1i2_nu1.block(0)
trainidx = [0,3,5]
bsamples = block.samples

In [547]:
new_samples = StructureMap(rhoii1i2_nu1.block(0).samples[:128]["structure"])[1]

## set up NICE style feature computation

## encoder-decoder like architecture - nope

In [20]:
import equistore
import torch
import equistore.operations as operations
from tqdm import tqdm
torch.set_default_dtype(torch.float64)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  


class featDataset(torch.utils.data.Dataset):
    #Dataset class
    def __init__(self, feature_path, frames):
        #
        self.features = np.load(feature_path, mmap_mode = 'r')
        self.keys_features = equistore.io._labels_from_npz(self.features["keys"])
        self.currentkey = 0
        self.frames = frames

    def __len__(self):
        return len(self.frames)
    
    def __getitem__(self, structure_idx):
        feature_block, feature_key = self.generate_feature_block(self.features, structure_idx)        
        #samples_filter, target_block_samples = self.get_index_from_idx(self.target.block(self.currentkey).samples, structure_idx)
        structure = [frames[i] for i in structure_idx]
        #Modify feature_block to tensormap
        feature_map = TensorMap(feature_key, feature_block)
        return feature_map, structure

    
    def generate_feature_block(self, memmap, structure_idx):
        #Generate the block from npz file
        output = []
        if self.currentkey is None:
            feature_key = self.keys_features     
        else:
            feature_key = Labels(["order_nu", "inversion_sigma", "spherical_harmonics_l", "species_center", "species_neighbor_a", "species_neighbor_b"], 
                                 np.asarray(self.keys_features[self.currentkey].astype(list), dtype=np.int32).reshape(-1,6))

#             feature_key = [self.keys_features[self.currentkey]]
        for key in feature_key:
            block_index = list(self.keys_features).index(key)
            prefix = f"blocks/{block_index}/values"        
            block_samples = equistore.io._labels_from_npz(memmap[f"{prefix}/samples"])
            block_components = []
            for i in range(1):
                block_components.append(equistore.io._labels_from_npz(memmap[f"{prefix}/components/{i}"]))
            block_properties = equistore.io._labels_from_npz(memmap[f"{prefix}/properties"])
             

            samples_filter, block_samples = self.get_index_from_idx(block_samples, structure_idx)

            block_data = memmap[f"{prefix}/data"][samples_filter]
            block = TensorBlock(block_data, block_samples, block_components, block_properties)
            output.append(block)
        return output, feature_key
    
    def get_n_properties(self, memmap, key):
        block_index = list(self.keys_features).index(key)
        prefix = f"blocks/{block_index}/values"  
        block_properties = equistore.io._labels_from_npz(memmap[f"{prefix}/properties"])
        
        return len(block_properties)
    
    def get_index_from_idx(self, block_samples, structure_idx):
        #Get samples label from IDX
        samples = Labels(names = ['structure'], values = np.array(structure_idx).reshape(-1,1))
        all_samples = block_samples[['structure']].tolist()
        set_samples_to_slice = set(samples.tolist())
        samples_filter = np.array(
            [sample in set_samples_to_slice for sample in all_samples]
        )
        new_samples = block_samples[samples_filter]
        
        return samples_filter, new_samples
    
    def collate_output_values(blocks):
        feature_out = []
        for sample_output in blocks:
            feature_block, structure = sample_output
            for z in feature_block:
                feature_out.append(torch.tensor(z.values))

        return feature_out


In [21]:
def mse_loss(pred, target):
    return torch.mean((pred - target)**2)


class BlockEncoderDecoder(torch.nn.Module):
    def __init__(self, seed=123, **kwargs):
        super().__init__()
        self.encoder_hidden_layer = torch.nn.Linear(
            in_features=kwargs["input_size"], out_features=kwargs["hidden_size"]
        )
        self.encoder_output_layer = torch.nn.Linear(
            in_features=kwargs["hidden_size"], out_features=kwargs["intermediate_size"]
        )
        self.decoder_hidden_layer = torch.nn.Linear(
            in_features=kwargs["intermediate_size"], out_features=kwargs["hidden_size"]
        )
        self.decoder_output_layer = torch.nn.Linear(
            in_features=kwargs["hidden_size"], out_features=kwargs["input_size"]
        )
        self.to(device)
        
    def forward(self, x):
        print("from block forward", x.keys)
        x = torch.from_numpy(x.block().values)
        activation = self.encoder_hidden_layer(x)
        activation = torch.relu(activation)
        code = self.encoder_output_layer(activation)
        code = torch.relu(code)
        activation = self.decoder_hidden_layer(code)
        activation = torch.relu(activation)
        activation = self.decoder_output_layer(activation)
        reconstructed = torch.relu(activation)
        return reconstructed

    
    def fit(self,traindata_loader, loss_function, optimizer_type, lr, reg, n_epochs):
        if optimizer_type == "Adam":
            optimizer = torch.optim.Adam(self.parameters(), lr = lr, weight_decay = reg.item())
            threshold = 200
            scheduler_threshold = 50
            tol = 0
            history_step = 1000
        
        elif optimizer_type == "LBFGS":
            optimizer = torch.optim.LBFGS(self.parameters(), lr = lr)
            threshold = 30
            scheduler_threshold = 10
            tol = 0
            history_step = 10
            
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, scheduler_threshold, gamma = 0.5)
        best_state = copy.deepcopy(self.state_dict())
        lowest_loss = torch.tensor(9999)
        pred_loss = torch.tensor(0)
        trigger = 0
        loss_history = []
        pbar = tqdm(range(n_epochs))
        
        for epoch in pbar:
            pbar.set_description(f"Epoch: {epoch}")
            pbar.set_postfix(pred_loss = pred_loss.item(), lowest_loss = lowest_loss.item(), trigger = trigger)
            
            for x_data, structure in traindata_loader: 
                optimizer.zero_grad()
                #x_data, y_data = x_data.to(self.device), y_data.to(self.device)
                if optimizer_type == "LBFGS":
                    def closure():
                        optimizer.zero_grad()
                        _pred = self.forward(x_data)                                        
                        _pred_loss = loss_function(_pred, x_data.block().values)
                        _pred_loss = torch.nan_to_num(_pred_loss, nan=lowest_loss.item(), posinf = lowest_loss.item(), neginf = lowest_loss.item())                 
                        _reg_loss = self.get_regression_values(reg.item()) #Only works for 1 layer
                        _new_loss = _pred_loss + _reg_loss
                        _new_loss.backward()
                        return _new_loss
                    optimizer.step(closure)

                elif optimizer_type == "Adam":
                    pred = self.forward(x_data)
#                     print(pred, x_data.block().values )
                    pred_loss = loss_function(pred, torch.from_numpy(x_data.block().values))
                    
                    #reg_loss = self.get_regression_values(reg.item())#Only works for 1 layer
                    new_loss = pred_loss #+ reg_loss
                    new_loss.backward()

                    optimizer.step()
                
            with torch.no_grad():
                current_loss = 0 
                for x_data, structure in traindata_loader:
                    pred = self.forward(x_data)
                    current_loss  += loss_function(pred, torch.from_numpy(x_data.block().values)) #Loss should be normalized already
                pred_loss = current_loss
                reg_loss = self.get_regression_values(reg.item()) 
                new_loss = pred_loss + reg_loss
                if pred_loss >100000 or (pred_loss.isnan().any()) :
                    print ("Optimizer shows weird behaviour, reinitializing at previous best_State")
                    self.load_state_dict(best_state)
                    if optimizer_type == "Adam":
                        optimizer = torch.optim.Adam(self.parameters(), lr = lr, weight_decay = reg.item())
                    elif optimizer_type == "LBFGS":
                        optimizer = torch.optim.LBFGS(self.parameters(), lr = lr)

                if epoch % history_step == 1:
                    loss_history.append(lowest_loss.item())
                
                if lowest_loss - new_loss > tol: #threshold to stop training
                    best_state = copy.deepcopy(self.state_dict())
                    lowest_loss = new_loss 
                    trigger = 0    
                    
                else:
                    trigger += 1
                    scheduler.step()
                    if trigger > threshold:
                        self.load_state_dict(best_state)
                        print ("Implemented early stopping with lowest_loss: {}".format(lowest_loss))
                        return loss_history
        return loss_history
    
    def get_regression_values(self, reg_weights):
        output = []
        for param in self.parameters():
            output.append(torch.sum(torch.pow(param,2)))
        try:
            output = torch.sum(torch.tensor(output) * reg_weights)
        except:
            output = 0
        return output


    

class ThreeCenterModel(torch.nn.ModuleDict):
    
    def __init__(self, feature, hidden_size=128, intermediate_size=18):
        super().__init__()
        self.feature = feature
        self.loss_history={}
        self.models = torch.nn.ModuleDict()
        for k, b in self.feature:
            input_size = b.values.shape[-1]
#             print(k,input_size, hidden_size, intermediate_size)
            self.models[str(k)] = BlockEncoderDecoder(input_size=input_size, hidden_size=hidden_size, intermediate_size=intermediate_size)            
        self.to(device)
    
    def forward(self, feature):
        pred_blocks =[]
        for k, block in feature:
#             print(self[str(k)])
            
            recons = self.models[str(k)].forward(block) # expects a tensorMap
            
            newblock = TensorBlock(
                        values=recons.reshape(block.values.shape),
                        samples=block.samples,
                        components=block.components,
                        properties= block.properties
                    )
            pred_blocks.append(newblock) 
        
        keys = feature.keys
        pred_target = TensorMap(keys, pred_blocks)
        return(pred_target)
    
    def fit(self, train_dataloader, regularization_dict, optimizer_type, n_epochs, loss_function, lr):
        #Iterates through the keys of self.model, then for each key we will fit self.model[key] with data[key]
        total = len(self.models)
        for index, key in enumerate(self.feature.keys):
            print ("Now training on Block {} of {}".format(index, total))
            train_dataloader.dataset.currentkey = index
            
            loss_history_key = self.models[str(key)].fit(train_dataloader, loss_function, optimizer_type, lr, regularization_dict[str(key)], n_epochs)

            self.loss_history[str(key)] = loss_history_key

    
    


In [22]:
regularization_dict={}
for key in rhoii1i2_nu1.keys:
    regularization_dict[str(key)] = torch.tensor(0)

In [23]:
def collate_blocks(block_tuple):
    feature_tensor_map, structure_array = block_tuple[0]
    return feature_tensor_map, structure_array

train_feature_path ="./rhoii1i2n1_400.npz"
train = featDataset(train_feature_path, frames[:400])
train_Sampler = torch.utils.data.sampler.SequentialSampler(train)
train_BSampler = torch.utils.data.sampler.BatchSampler(train_Sampler, batch_size = 500, drop_last = False)

test_feature_path ="./rhoii1i2n1_400-500.npz"
test = featDataset(train_feature_path, frames[400:500])
test_Sampler = torch.utils.data.sampler.SequentialSampler(test)
test_BSampler = torch.utils.data.sampler.BatchSampler(test_Sampler, batch_size = 500, drop_last = False)

train_loader = torch.utils.data.DataLoader(train, sampler = train_BSampler, collate_fn = collate_blocks)
test_loader = torch.utils.data.DataLoader(test, sampler = test_BSampler, collate_fn = collate_blocks)

In [24]:
# rhoii1i2_nu1.keys[
rhoii1i2_nu1.blocks_matching(species_center=3)
# ]

[9, 10, 11, 12, 13, 14, 15, 16, 17]

In [25]:
model = ThreeCenterModel(rhoii1i2_nu1)

In [26]:
model.fit(train_loader, regularization_dict, optimizer_type="Adam", n_epochs=2, loss_function=mse_loss, lr=0.01)

Now training on Block 0 of 27


Epoch: 0:   0%|                                                                                              | 0/2 [00:00<?, ?it/s, lowest_loss=9999, pred_loss=0, trigger=0]

from block forward [(3, 1, 0, 1, 1, 1)]
from block forward [(3, 1, 0, 1, 1, 1)]


Epoch: 1: 100%|███████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.94it/s, lowest_loss=0.000316, pred_loss=0.000316, trigger=0]


from block forward [(3, 1, 0, 1, 1, 1)]
from block forward [(3, 1, 0, 1, 1, 1)]
Now training on Block 1 of 27


...

Epoch: 0:   0%|                                                                                              | 0/2 [00:00<?, ?it/s, lowest_loss=9999, pred_loss=0, trigger=0]

from block forward [(3, 1, 0, 12, 12, 12)]
from block forward [(3, 1, 0, 12, 12, 12)]


Epoch: 1:  50%|█████████████████████████████████████▌                                     | 1/2 [00:19<00:19, 19.76s/it, lowest_loss=0.000311, pred_loss=0.000311, trigger=0]

from block forward [(3, 1, 0, 12, 12, 12)]
from block forward [(3, 1, 0, 12, 12, 12)]


Epoch: 1: 100%|███████████████████████████████████████████████████████████████████████████| 2/2 [00:39<00:00, 19.59s/it, lowest_loss=0.000311, pred_loss=0.000311, trigger=0]


In [28]:
for x_data, structures in test_loader:
#     print(x_data)
    pred = model(x_data)
#     print ("Test error for mse_full is {}".format(mse_full(pred, )))

AttributeError: 'TensorBlock' object has no attribute 'keys'

In [ ]:
# aa=[]
# bb=[]
# for x, y in train_loader:
#     aa.append(x)
#     bb.append(y)
# from itertools import chain
# list(chain.from_iterable(bb))==frames[:400]

In [10]:
rascal_hypers = {
    "interaction_cutoff": 4.5,
    "cutoff_smooth_width": 0.3,
    "max_radial": 4,
    "max_angular": 2,
    "gaussian_sigma_type": "Constant",
    "compute_gradients":  False,
#     "expansion_by_species_method": "user defined",
#     "global_species": [1,6,8,7]
    
}

cg = ClebschGordanReal(5)

spex = RascalSphericalExpansion(rascal_hypers)
rhoi = spex.compute(frames)
pairs = RascalPairExpansion(rascal_hypers)
gij = pairs.compute(frames)

### $\rho_{i}^{\otimes 3}$

In [23]:
rho1i = acdc_standardize_keys(rhoi)
rho1i = rho1i.keys_to_properties(['species_neighbor'])

rho2i = cg_increment(rho1i, rho1i, lcut=4, other_keys_match=["species_center"], clebsch_gordan=cg)
rho3i =  cg_increment(rho2i, rho1i, lcut=0, other_keys_match=["species_center"], clebsch_gordan=cg)

### $\rho_{i i_1 i_2}^{\otimes 1}$

In [26]:
gij =  acdc_standardize_keys(gij)
rhoii1i2_nu0 = cg_combine(gij, gij, clebsch_gordan=cg,lcut=4, other_keys_match=['species_center'])
rhoii1i2_nu1 =  cg_combine(rho1i, rhoii1i2_nu0, clebsch_gordan=cg, lcut=0, other_keys_match = ['species_center'])


### check degeneracy of $\rho_{i}^{\otimes 3}$

In [30]:
np.linalg.norm(rho3i.block(species_center = 3, inversion_sigma = 1).values[0] - rho3i.block(species_center = 3, inversion_sigma = 1).values[1])

1.680772513045776e-22

In [29]:
# rhoii1i2_nu1.keys_to_properties('species_neighbor_1')
# rhoii1i2_nu1.keys_to_properties('species_neighbor_2')

In [49]:
rhoii1i2_nu1.block(species_center = 3,species_neighbor_a=12, species_neighbor_b=12, inversion_sigma=1).samples[36:72]#values.shape


Labels([(1, 0, 1, 1), (1, 0, 1, 2), (1, 0, 1, 3), (1, 0, 1, 4),
        (1, 0, 1, 5), (1, 0, 1, 6), (1, 0, 2, 1), (1, 0, 2, 2),
        (1, 0, 2, 3), (1, 0, 2, 4), (1, 0, 2, 5), (1, 0, 2, 6),
        (1, 0, 3, 1), (1, 0, 3, 2), (1, 0, 3, 3), (1, 0, 3, 4),
        (1, 0, 3, 5), (1, 0, 3, 6), (1, 0, 4, 1), (1, 0, 4, 2),
        (1, 0, 4, 3), (1, 0, 4, 4), (1, 0, 4, 5), (1, 0, 4, 6),
        (1, 0, 5, 1), (1, 0, 5, 2), (1, 0, 5, 3), (1, 0, 5, 4),
        (1, 0, 5, 5), (1, 0, 5, 6), (1, 0, 6, 1), (1, 0, 6, 2),
        (1, 0, 6, 3), (1, 0, 6, 4), (1, 0, 6, 5), (1, 0, 6, 6)],
       dtype=[('structure', '<i4'), ('center', '<i4'), ('neighbor_1', '<i4'), ('neighbor_2', '<i4')])

In [46]:
np.linalg.norm(rhoii1i2_nu1.block(species_center = 3,species_neighbor_a=1, species_neighbor_b=12, inversion_sigma=1 ).values[:36])

2.5486800904721503e-07